## RNN for PTBDB dataset
We tried multiple architectures (both shallow and deeper), and multiple values for the hyperparameters through [hyperparameter-tuning](hyperparameter-tuning_ptbdb.py), but a regular RNN with either a LSTM or a GRU cell couldn't learn anything meaningful from the data. The PTBDB dataset is small, and the networks get stuck predicting only one class.

For demonstration, here are the results from a small RNN with 2 GRU layers and a single Fully Connected layer. 

In [1]:
import pandas as pd
import numpy as np

import tensorflow

from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, GRU, \
    concatenate, Add, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
from IPython.display import Image

### Import data

In [2]:
df_1 = pd.read_csv("../data/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("../data/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

train = False

### Load model and skip to [evaluation](#evaluate)

In [3]:
model = load_model("gru_ptbdb_model")

### Or ALTERNATIVELY train it

In [4]:
seq_len = 187

def get_model():
    n_class = 1
    inp = Input(shape=(187, 1))
    x = GRU(128,
           return_sequences=True,
           dropout=0.2)(inp)
    x = GRU(128,
           return_sequences=False,
           dropout=0.2)(x)
    
    dense = Dense(64, activation='relu')(x)
    dense = Dropout(0.2)(dense)
    output = Dense(n_class, activation='sigmoid')(dense)
    
    opt = tensorflow.keras.optimizers.Adam(lr=0.001)
    
    model = models.Model(inputs=inp, outputs=output)
    class_weights = compute_class_weight('balanced', [0, 1], Y)

    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    return model

In [5]:
if train:
    model = get_model()
    file_path = "rnn_ptbdb_retrain.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1)
    redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
    callbacks_list = [checkpoint, early, redonplat]  # early

    model.fit(X, Y, epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)
    model.load_weights(file_path)

### Evaluate
<a id='evaluate'></a>
The model predicts the same class for every sample.

In [6]:
pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

auc_roc = roc_auc_score(Y_test, pred_test)

print("AUROC score : %s "% auc_roc)

precision, recall, _ = precision_recall_curve(Y_test, pred_test)

auc_prc = auc(recall, precision)
print("AUPRC score : %s "% auc_prc)

print(confusion_matrix(Y_test, pred_test))

Test f1 score : 0.8386195890684222 
Test accuracy score : 0.7220886293369976 
AUROC score : 0.5 
AUPRC score : 0.8610443146684987 
[[   0  809]
 [   0 2102]]
